In [6]:
!apt update -y -qq
!apt install -y -qq curl lshw libcairo2-dev pkg-config python3-dev
!curl https://ollama.ai/install.sh | sh

import subprocess
sub = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE)

56 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
lshw is already the newest version (02.19.git.2021.06.19.996aaad9c7-2build1).
pkg-config is already the newest version (0.29.2-1ubuntu3).
curl is already the newest version (7.81.0-1ubuntu1.19).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  26023   

In [ ]:
# !pip install colab-xterm
# %load_ext colabxterm

In [ ]:
# %xterm

In [7]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 6a0746a1ec1a... 100% ▕▏ 4.7 GB                         
pulling 4fa551d4f938... 100% ▕▏  12 KB                         
pulling 8ab4849b038c... 100% ▕▏  254 B                         
pulling 577073ffcc6c... 100% ▕▏  110 B                         
pulling 3f8eb4da87fa... 100% ▕▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [1]:
# !pip install autogen-agentchat~=0.2 groq sacrebleu
!pip install sacrebleu
!pip install 'autogen-agentchat==0.4.0.dev11' 'autogen-ext[openai]==0.4.0.dev11'

In [5]:
import copy
import json
import os
import numpy
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union

from openai import BadRequestError

# import autogen
# from autogen_agentchat.agents import UserProxyAgent, AssistantAgent, BaseChatAgent
from autogen import AssistantAgent, UserProxyAgent, ConversableAgent
# from autogen_agentchat.agents import config_list_from_json
from autogen.agentchat import Agent
# from autogen.agentchat.contrib.agent_optimizer import AgentOptimizer
# from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent
# from autogen.code_utils import extract_code
# from autogen.math_utils import get_answer

## Data

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
def read_file(url):
    file = open(url, "r")
    data = file.read().split('\n')
    return data

In [30]:
data_url = 'drive/MyDrive/uploads/'

phoMT_dev_en = read_file(data_url + "data/PhoMT/detokenization/dev/dev.en")
phoMT_dev_vi = read_file(data_url + "data/PhoMT/detokenization/dev/dev.vi")
phoMT_test_en = read_file(data_url + "data/PhoMT/detokenization/test/test.en")
phoMT_test_vi = read_file(data_url + "data/PhoMT/detokenization/test/test.vi")
phoMT_train_en = read_file(data_url + "data/PhoMT/detokenization/train/train.en")
phoMT_train_vi = read_file(data_url + "data/PhoMT/detokenization/train/train.vi")

In [31]:
phoMT_dev_en[0] = phoMT_dev_en[0][1:]

In [32]:
phoMT_dev_envi = [];
for index in range(len(phoMT_dev_en)):
    phoMT_dev_envi.append({"question":phoMT_dev_en[index], "answer": phoMT_dev_vi[index]})
phoMT_test_envi = [];
for index in range(len(phoMT_test_en)):
    phoMT_test_envi.append({"question": phoMT_test_en[index], "answer": phoMT_test_vi[index]})
phoMT_train_envi = [];
for index in range(len(phoMT_train_en)):
    phoMT_train_envi.append({"question": phoMT_train_en[index], "answer": phoMT_train_vi[index]})
# phoMT_dev_envi

In [33]:
phoMT_dev_envi[0]["question"]

'Hurricane Dorian, one of the most powerful storms ever recorded in the Atlantic Ocean, made landfall as a Category 5 storm on Great Abaco Island in the northern Bahamas on Sunday morning, September 1, 2019.'

## Benchmark

### SacreBleu

In [34]:
from sacrebleu.metrics import BLEU, CHRF, TER
bleu = BLEU()

def get_score(src, ans, res):
     return bleu.corpus_score([ans], [[res]]).score

## Agent init

### Agents declarations

In [8]:
llm_config = {
    "config_list": [
        {
            "model": "llama3",
            "base_url": "http://localhost:11434/v1",
            "api_key": "ollama",
        }
        # {
        #     "api_type": "groq",
        #     "model": "llama3-8b-8192",
        #     "api_key": "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",
        # }
        # {
        #     "model": "meta-llama/llama-3.1-405b-instruct:free",
        #     "base_url": "https://openrouter.ai/api/v1",
        #     "api_key": "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"
        # }
    ]
}
# open router: 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

In [13]:
# import requests
# import json
# import time

In [14]:
# response = requests.post(
#   url="https://openrouter.ai/api/v1/chat/completions",
#   headers={
#     "Authorization": f"Bearer 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",
#   },
#   data=json.dumps({
#     "model": "meta-llama/llama-3.1-8b-instruct:free", # Optional
#     "messages": [
#         {
#             "role": "user",
#             "content": """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
#     Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
#     The text:
#     At this time, there have been no reported injuries among the 46 publishers in the two congregations on Great Abaco Island.

# Respond with only the improved translation, without any additional explanations or commentary."""
#         },
#     ]
#   })
# )

In [15]:
# response.json()["choices"][0]["message"]["content"]

### Custom UserProxyAgent

In [36]:
class JudgeProxyAgent(UserProxyAgent):
    MAX_CONSECUTIVE_AUTO_REPLY = 10
    DEFAULT_REPLY_TEMPLATE = "Generate a response more closely resembling the style, detail, and tone of the provided answer. Focus on specifying key elements to capture the nuances of this answer effectively. The answer: "
    PROMPTS = """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    """

    def __init__(
        self,
        name: Optional[str] = "JudgeChatAgent",
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        **kwargs,
    ):
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self.register_reply(
            trigger=ConversableAgent, reply_func=JudgeProxyAgent._check_final_result, position=0
        )
        self.max_function_call_trial = 3
        self.query = None
        self._answer = None
        self.is_correct = None
        self.previous = None
        # self.history = []

    def initiate_chat(
        self,
        recipient,
        history: List[Dict] = [],
        answer=None,
        previous=None,
        silent: Optional[bool] = False,
        max_turns = 3,
        **context,
    ):
        self.query = context["message"]
        self._answer = answer
        self.previous = previous

        self.is_correct = None
        self.max_function_call_trial = max_turns
        recipient.history = history
        print("Recipient.history: " + str(len(recipient.history)))

        self._prepare_chat(recipient, True)

        error_message = None
        try:
            prompt = self.PROMPTS + context['message']
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            self.is_correct = 0
            print("error information: {}".format(error_message))

        recipient.reset()
        self.is_correct = copy.deepcopy(self.is_correct)
        result = self.is_correct
        self._reset()
        return result

    def receive(
        self,
        message: Union[Dict, str],
        sender: Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        self._process_received_message(message, sender, silent)
        if request_reply is False or request_reply is None and self.reply_at_receive[sender] is False:
            return

        self.is_correct = self.chat_messages[sender][-1].get("content")
        if self._answer is not None:
            if (get_score(self.query, self.is_correct, self._answer) >= 90):
                return

        self.max_function_call_trial = self.max_function_call_trial - 1
        if (self.max_function_call_trial <= 0):
            self.max_function_call_trial = 0
            return

        # reply = f"Using the original sentence: {self.query}, provide guidance to improve the quality of {self.is_correct}. Focus on enhancing accuracy, tone, fluency, and contextual appropriateness."
        reply = f"Translate the sentence: {self.query} into Vietnamese, ensuring that when the translation is appended to the previous translated paragraph: {self.previous}, the entire paragraph remains coherent, meaningful, and contextually appropriate. Respond with only the translated sentence, without any additional commentary or explanation."
        if self._answer is not None:
            reply = f'Analyze the original sentence: {self.query}, the expected Vietnamese translation: {self._answer}, and the generated translation: {self.is_correct}. Identify the differences between {self.is_correct} and {self._answer}, and provide guidance to improve the translation so it aligns more closely with {self._answer}. Focus on preserving meaning, tone, style, and naturalness in Vietnamese while addressing any discrepancies.'
        # if self._answer is not None:
        #     reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
        self.send(reply, sender, silent=silent)

    def _check_final_result(
        self,
        messages: Optional[List[Dict]] = None,
        sender: Optional[Agent] = None,
        config: Optional[Any] = None,
    ):
        messages = messages[-1]
        if isinstance(messages, dict):
            messages = messages.get("content")
            if messages is None:
                return False, None
            if (messages.find("\n") >= 0):
                print("Response longer than expected?\n" + messages)
                # messages = messages.split("\n")[0]

        self.is_correct = messages
        temp_score = get_score(self.query, messages, self._answer)
        print("Score: " + str(temp_score))
        if (temp_score >= 90):
            return True, "The result is passable. Please reply me with the same answer as before."
        return False, None

    def _reset(self):
        # super()._reset()
        self.max_function_call_trial = 0
        self.is_correct = None
        self.query = None
        self._answer = None
        self.previous = None

In [10]:
# def generate_oai_reply_openrouter(messages):
#     response = requests.post(
#             url="https://openrouter.ai/api/v1/chat/completions",
#             headers={
#                 "Authorization": f"Bearer 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",
#             },
#             data=json.dumps({
#                 "model": "meta-llama/llama-3.1-8b-instruct:free", # Optional
#                 "messages": messages
#             })
#         )
#     if (not response.json().get("choices")):
#         time.sleep(60)
#         response = requests.post(
#             url="https://openrouter.ai/api/v1/chat/completions",
#             headers={
#                 "Authorization": f"Bearer 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",
#             },
#             data=json.dumps({
#                 "model": "meta-llama/llama-3.1-8b-instruct:free", # Optional
#                 "messages": messages
#             })
#         )
#         if (not response.json().get("choices")):
#             return None
#     return response.json()["choices"][0]["message"]["content"]

In [24]:
class PromptAssistant(AssistantAgent):
    PROMPTS = """Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    """
    TRIMMING_PROMPT = "Respond with only the improved translation, without any additional explanations or commentary."
    def __init__(
        self,
        after: Agent,
        before: Agent = None,
        name: Optional[str] = "PromptAssistantAgent",
        human_input_mode: Literal["ALWAYS", "NEVER", "TERMINATE"] = "NEVER",
        **kwargs,
    ):
        super().__init__(
            name=name,
            human_input_mode=human_input_mode,
            **kwargs,
        )
        self.before = before
        self.after = after
        self.history = []
        self.is_start = False
        self.last_msg = "NaH"

    def initiate_chat(
        self,
        recipient,
        history: List[Dict] = None,
        silent: Optional[bool] = False,
        max_turns = 3,
        **kwargs
    ):
        self.is_start = True
        self._prepare_chat(recipient, True)
        for msg in history:
            self._append_oai_message(
                message=msg['content'],
                role=msg['role'],
                conversation_id = recipient
            )
        print(len(self.chat_messages_for_summary(recipient)))
        error_message = None
        try:
            prompt = self.PROMPTS + kwargs['message']
            self.send(prompt, recipient, silent=silent)
        except BadRequestError as e:
            error_message = str(e)
            print("error information: {}".format(error_message))
        recipient.reset()
        self.is_start = False
        print(self.chat_messages_for_summary(recipient))
        return self.chat_messages_for_summary(recipient)[-1]['content']

    def receive(
        self,
        message: Union[Dict, str],
        sender: Agent,
        request_reply: Optional[bool] = None,
        silent: Optional[bool] = False,
    ):
        self._process_received_message(message, sender, silent)
        if request_reply is False or request_reply is None and self.reply_at_receive[sender] is False:
            return
        if self.is_start:
            # reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            # if reply is not None:
            #     self.send(reply, sender, silent=silent)
            return
        if (sender == self.before):
            reply = sender.chat_messages_for_summary(self)[0]['content']
            if len(sender.chat_messages_for_summary(self)) > 1:
                # state, res = self.generate_oai_reply(messages=self.chat_messages[sender], sender=sender)
                # if state:
                #     reply = res
                reply = self.generate_reply(messages=self.chat_messages[sender], sender=sender)
            else:
                print("Self.history: " + str(len(self.history)))
                for msg in self.history:
                    self._append_oai_message(
                        message=msg['content'],
                        role=msg['role'],
                        conversation_id = self.after,
                        is_sending = (msg['role'] != 'user')
                    )
            if reply is not None:
                if reply is str:
                    reply = reply + "\n\n" + self.TRIMMING_PROMPT
                else:
                    reply = str(reply) + "\n\n" + self.TRIMMING_PROMPT
                self.last_msg = reply
                # print(self.last_msg)
                self.send(reply, self.after, silent=silent, request_reply=True)
                if (self._oai_messages[self.after][-1]['content'].find('Note') != -1):
                    temp = self._oai_messages[self.after][-1]['content']
                    self._oai_messages[self.after][-1]['content'] = temp[:temp.find('Note')].strip()
                # print(self.chat_messages_for_summary(self.after))
                self.history = self.chat_messages_for_summary(self.after)
                self.send(self.chat_messages_for_summary(self.after)[-1]['content'], self.before, silent=silent)

In [25]:
# class LLMAssistant(AssistantAgent):
#     def _generate_oai_reply_from_client(self, llm_client, messages, cache) -> Union[str, Dict, None]:
#         extracted_response = generate_oai_reply_openrouter(messages)
#         if extracted_response is None:
#             print(f"Extracted_response from {response} is None.")
#             # warnings.warn(f"Extracted_response from {response} is None.", UserWarning)
#             return None
#         return extracted_response

In [38]:
LLM = AssistantAgent(
    name="LLM",
    system_message="You are a helpful assistant",
    code_execution_config=False,
    llm_config=llm_config,
)

Judge = JudgeProxyAgent(
    name="Judge",
    system_message="You are an advisor",
    code_execution_config=False,
)

PromptGenerator = PromptAssistant(
    before=Judge,
    after=LLM,
    name="PromptGenerator",
    system_message="You are a prompt engineer",
    human_input_mode = "NEVER",
    code_execution_config=False,
    llm_config=llm_config,
)

In [39]:
result = Judge.initiate_chat(
    recipient = PromptGenerator,
    max_turns = 2,
    message = phoMT_dev_envi[4]["question"],
    answer = phoMT_dev_envi[4]["answer"]
)

Recipient.history: 0


AttributeError: 'JudgeProxyAgent' object has no attribute 'send'

In [ ]:
len(PromptGenerator.history)
    # print(Judge.chat_messages(PromptGenerator)[x])

4

In [ ]:
len(PromptGenerator.chat_messages_for_summary(LLM))

0

In [ ]:
# print(result)

In [ ]:
# judge_result = "Tới giờ, chưa có báo cáo về nạn nhân thương tật nào trong số 46 nhà xuất bản tại hai giáo xứ trên đảo Abaco Đại."
# get_score(phoMT_dev_envi[4]["question"], result, phoMT_dev_envi[4]["answer"])

NameError: name 'result' is not defined

In [ ]:
# result = PromptGenerator.initiate_chat(
#     recipient = LLM,
#     max_turns = 2,
#     message = phoMT_dev_envi[4]["question"],
#     history = PromptGenerator.history,
#     clear_history = False
# )

In [ ]:
def score_translate(message, answer, turns = 1, history = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            answer = answer,
            history = history
        )
    return round(get_score(message, str(result), answer) / 100.0, 2)

def score_translate_test(message, answer, turns = 1, history = None, context = None):
    if history is None:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            previous = context
        )
    else:
        result = Judge.initiate_chat(
            recipient = PromptGenerator,
            max_turns = turns + 1,
            message = message,
            history = history,
            previous = context
        )
    # result = PromptGenerator.initiate_chat(
    #     recipient = LLM,
    #     max_turns = turns + 1,
    #     message = message,
    #     history = history,
    #     clear_history = False
    # )
    # return result
    return round(get_score(message, str(result), answer) / 100.0, 2)

# def score_translate_comet(message, answer, turns = 1, history = None):
#     if history is None:
#         result = Judge.initiate_chat(
#             recipient = PromptGenerator,
#             max_turns = turns + 1,
#             message = message,
#             answer = answer,
#         )
#     else:
#         result = Judge.initiate_chat(
#             recipient = PromptGenerator,
#             max_turns = turns + 1,
#             message = message,
#             answer = answer,
#             history = history
#         )
#     return round(get_score_comet(message, str(result), answer))

# def score_translate_comet_test(message, answer, turns = 1, history = None, context = None):
#     if history is None:
#         result = Judge.initiate_chat(
#             recipient = PromptGenerator,
#             max_turns = turns + 1,
#             message = message,
#             previous = context
#         )
#     else:
#         result = Judge.initiate_chat(
#             recipient = PromptGenerator,
#             max_turns = turns + 1,
#             message = message,
#             history = history,
#             previous = context
#         )
#     return round(get_score_comet(message, str(result), answer))

## Improve

### Before

In [ ]:
len(phoMT_test_envi)

19152

In [ ]:
train_num = 100
test_num = 50
batch = 10
begin = 0

In [ ]:
# score_test = []
# for i in range(test_num):
#     question = " ".join([x['question'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
#     answer = " ".join([x['answer'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
#     # print(question)
#     # print(answer)
#     # print()
#     score_test.append(score_translate(
#         message= question,
#         answer= answer,
#         turns= 0
#     ))

In [ ]:
# numpy.average(score_test)

In [ ]:
# score_test

## Learn

In [ ]:
len(phoMT_train_envi)

2978000

In [ ]:
EPOCH = 1
turns = 1
# optimizer_model = "gpt-4-1106-preview"
# optimizer = AgentOptimizer(max_actions_per_step=3, llm_config=llm_config, optimizer_model="llama3")
history_recorder = []
last_msg_s = []
for index in range(train_num):
    # query = phoMT_test_envi[index]
    question = " ".join([x['question'] for x in phoMT_test_envi[begin + train_num * batch + index * batch : begin + train_num * batch + (index+1) * batch]])
    answer = " ".join([x['answer'] for x in phoMT_test_envi[begin + train_num * batch + index * batch : begin + train_num * batch + (index+1) * batch]])
    # is_correct = user_proxy.initiate_chat(assistant, answer=query["answer"], problem=query["question"])
    result = score_translate(question, answer, turns)
    history = PromptGenerator.history
    print(f"Test: {index}")
    history_recorder.extend(history)
    last_msg_s.append(PromptGenerator.last_msg)
    print(len(history_recorder))

Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    Radio waves have long wavelengths, while x - rays have short wavelengths. And visible light, the kind you can actually see, is somewhere in between. Even though our eyes ca n't detect light outside of the visible range, we can build special detectors that are stimulated by these other wavelengths of light, kind of like digital eyes. With these devices, we can measure the light that is there, even though we ca n't see it ourselves. Even though they seem different, the warmth you feel from a crackling fire

RuntimeError: Groq exception occurred: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [ ]:
len(history_recorder)

In [ ]:
# Save the history to .json file for future usage
with open(data_url + f'data/{train_num}_{test_num}_{batch}_{begin}.json', 'w', encoding='utf-8') as f:
    json.dump(history_recorder, f, ensure_ascii=False, indent=4)

## Test

In [ ]:
# Load the history from data.json file
with open(data_url + f'data/{train_num}_{test_num}_{batch}_{begin}.json', 'r') as file:
    history_recorder = json.load(file)

In [ ]:
score_test = []
answers = []
for i in range(test_num):
    question = " ".join([x['question'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
    answer = " ".join([x['answer'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
    print("Test " + str(i) + ":")
    # context = answers.join(" ")
    context = " "
    if (len(answers) > 0): context = answers[-1]
    # if (len(answers)>5): context = answers[-10:].join(" ")
    score_test.append(score_translate_test(
        message = question,
        answer = answer,
        turns = 0,
        history = history_recorder,
        context = context
    ))

Test 0:
Recipient.history: 0
Judge (to PromptGenerator):

Translate a sentence from English to Vietnamese. Produce an accurate, context-sensitive translation that maintains the tone and meaning of the original sentence. Ensure that the output sounds natural for native Vietnamese speakers.
    Respond only with the requested output. Do not include any explanations, introductions, follow-up remarks, or additional feedback. Provide exactly and only what is specified in the task.
    The text:
    On July 17, 2017, a three - judge panel of the Appellate Chamber of Russia's Supreme Court upheld the Court's April 20 decision "to liquidate the religious organization 'Administrative Center of Jehovah's Witnesses in Russia' and the local religious organizations that are part of its structure to turn over to the Russian Federation all property of the liquidated religious organization." With this decision, the Court effectively banned the worship of Jehovah's Witnesses throughout Russia. The foll

In [ ]:
numpy.average(score_test)

0.16

In [ ]:
score_test

[0.0, 0.2, 0.26, 0.0, 0.24, 0.26]

In [ ]:
# score_test = []
# answers = []
# for i in range(test_num):
#     question = " ".join([x['question'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
#     answer = " ".join([x['answer'] for x in phoMT_test_envi[train_num * batch + i * batch : train_num * batch + (i+1) * batch]])
#     print("Test " + str(i) + ":")
#     context = " "
#     # context = answers.join(" ")
#     if (len(answers) > 0): context = answers[-1]
#     # if (len(answers)>5): context = answers[-10:].join(" ")
#     score_test.append(score_translate_test(
#         message = question,
#         answer = answer,
#         turns = 1,
#         history = history_recorder,
#         context = context
#     ))

In [ ]:
# numpy.average(score_test)

In [ ]:
# score_test = []
# answers = []
# for i in range(test_num):
#     x = phoMT_test_envi[train_num+i]
#     print("Test " + str(i) + ":")
#     context = " ".join(answers)
#     # if (len(answers)>5) context = answers[-5:].join(" ")
#     score_test.append(score_translate_comet_test(
#         message = x['question'],
#         answer = x['answer'],
#         turns = 0,
#         history = history_recorder,
#         context = context
#     ))

In [ ]:
# numpy.average(score_test)

In [ ]:
# score_test = []
# answers = []
# for i in range(test_num):
#     x = phoMT_test_envi[train_num+i]
#     print("Test " + str(i) + ":")
#     context = " ".join(answers)
#     # if (len(answers)>5) context = answers[-5:].join(" ")
#     score_test.append(score_translate_comet_test(
#         message = x['question'],
#         answer = x['answer'],
#         turns = 1,
#         history = history_recorder,
#         context = context
#     ))

In [ ]:
# numpy.average(score_test)